A changer data_dir

In [1]:
import os
import json
import shutil
import cv2

# Define paths
data_dir = "./labels-pals_all"
output_dir = 'dataset'  # Dossier de sortie principal
yolo5label_dir = os.path.join(output_dir, 'labels')  # Dossier de sortie pour les annotations YOLO
output_image_dir = os.path.join(output_dir, 'images')  # Dossier de sortie pour les images

# Create output directories if not exist
os.makedirs(yolo5label_dir, exist_ok=True)
os.makedirs(output_image_dir, exist_ok=True)

# Define classes
classes = ['label', 'pal']  # Update this list with your actual class names

# Helper function to convert bounding box coordinates
def convert_bbox(img_size, bbox):
    dw = 1.0 / img_size[1]  # width
    dh = 1.0 / img_size[0]  # height
    x_center = (bbox[0] + bbox[2]) / 2.0
    y_center = (bbox[1] + bbox[3]) / 2.0
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    return (x_center * dw, y_center * dh, width * dw, height * dh)

# Function to calculate bounding box from polygon points
def get_bounding_box_from_polygon(points):
    x_coords = [point[0] for point in points]
    y_coords = [point[1] for point in points]
    xmin = min(x_coords)
    ymin = min(y_coords)
    xmax = max(x_coords)
    ymax = max(y_coords)
    return [xmin, ymin, xmax, ymax]

# Iterate through JSON files
for file in os.listdir(data_dir):
    if file.endswith('.json'):
        json_path = os.path.join(data_dir, file)
        
        with open(json_path) as f:
            data = json.load(f)
            image_path = os.path.join(data_dir, data['imagePath'])
            
            # Check if the image exists
            if not os.path.isfile(image_path):
                print(f"Image not found for: {data['imagePath']}, skipping this JSON file.")
                continue
            
            img = cv2.imread(image_path)
            if img is None:
                print(f"Could not read image: {image_path}, skipping this JSON file.")
                continue
            
            # Get image dimensions
            img_height, img_width = img.shape[:2]
            
            # Prepare YOLO annotation data
            yolo_data = []
            for shape in data['shapes']:
                label = shape['label']
                if label not in classes:
                    continue
                class_id = classes.index(label)
                
                # Get the bounding box from polygon points
                points = shape['points']
                bbox = get_bounding_box_from_polygon(points)
                
                # Convert the bounding box to YOLO format
                yolo_bbox = convert_bbox((img_height, img_width), bbox)
                yolo_data.append(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")
            
            # Write YOLO file if there is valid data
            if yolo_data:
                # Save YOLO annotation file
                yolo_filename = os.path.splitext(file)[0] + '.txt'
                yolo_filepath = os.path.join(yolo5label_dir, yolo_filename)
                with open(yolo_filepath, 'w') as yolo_file:
                    yolo_file.writelines(yolo_data)
                
                # Copy image to output image directory
                shutil.copy(image_path, output_image_dir)

print("Finished processing JSON files. Images and labels are organized in 'dataset' folder.")


Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744332_1716744332_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744339_1716744339_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744347_1716744347_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744354_1716744354_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744362_1716744362_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744370_1716744370_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744377_1716744377_992_10.jpg, skipping this JSON file.
Image not found for: ..\passageDroite\passageDroite_Bloc1_Camera3_1716744385_1716744385_992_10.jpg, skipping this JSON file.


clustering

In [2]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model

# Définir le chemin du dossier contenant les images du dataset
image_folder = r"dataset/images"  # Dossier de vos images

# Charger le modèle ResNet50 pré-entraîné sans la dernière couche
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")
model = Model(inputs=base_model.input, outputs=base_model.output)

# Fonction pour extraire les features d'une image
def extract_features(image_path):
    # Charger et pré-traiter l'image
    image = load_img(image_path, target_size=(224, 224))  # Taille pour ResNet50
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)  # Ajouter une dimension pour le batch
    image = preprocess_input(image)  # Pré-traitement spécifique à ResNet50
    # Extraire les features avec le modèle
    features = model.predict(image)
    return features.flatten()

# Récupérer les chemins des images dans le dossier
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith((".jpg", ".png"))]

# Extraire les features pour toutes les images
features = np.array([extract_features(img) for img in image_paths])

# Enregistrer les features dans un fichier .npy pour les utiliser ultérieurement
np.save("dataset/image_features.npy", features)

print("Extraction des features terminée.")
print(f"Dimensions des features pour chaque image : {features.shape[1]}")
print(f"Nombre total d'images traitées : {features.shape[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [3]:
print (features)

from sklearn.decomposition import PCA
# Appliquer PCA avec 95% de variance expliquée
pca = PCA(n_components=0.95)
reduced_features = pca.fit_transform(features)

print("Dimensions après PCA :", reduced_features.shape)


[[1.5968913  0.39919692 0.37277555 ... 0.9559643  0.08350623 1.1602477 ]
 [2.0621529  0.38926405 0.20754331 ... 0.54535705 0.8660398  2.4449983 ]
 [1.4500078  0.2107307  0.12172031 ... 0.12902373 0.00643835 0.67959297]
 ...
 [0.         0.17727941 0.19513558 ... 0.6225666  0.4679885  0.14159922]
 [0.33738962 0.14756268 0.00998511 ... 0.34966508 0.0361148  0.02193781]
 [0.32573423 0.07811625 0.02427218 ... 0.77365285 0.5115431  0.09218398]]
Dimensions après PCA : (1813, 380)


In [4]:

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Définir le nombre de clusters
n_clusters = 7

# Appliquer KMeans sur les features réduites
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(reduced_features)
labels = kmeans.labels_

# Afficher les résultats
print("Clustering terminé.")
print(f"Labels de cluster pour chaque image : {labels}")



Clustering terminé.
Labels de cluster pour chaque image : [5 5 5 ... 1 1 1]


In [5]:
import os
import shutil
import random


# Dossier de destination des clusters
output_base_dir = os.path.join(output_dir, 'cluster')
os.makedirs(output_base_dir, exist_ok=True)
image_paths = [f for f in os.listdir(output_image_dir) if f.endswith(('.jpg', '.png'))]

# Crée un dictionnaire pour regrouper les images par clusters
clusters = {}
for img, cluster in zip(image_paths, labels):
    if cluster not in clusters:
        clusters[cluster] = []
    clusters[cluster].append(img)

# Fonction pour copier les fichiers en divisant en train, val et test pour chaque cluster
def copy_files_by_cluster(clusters_dict):
    for cluster_label, image_list in clusters_dict.items():
        # Crée les dossiers pour chaque cluster
        cluster_dir = os.path.join(output_base_dir, f"cluster_{cluster_label}")
        train_img_dir = os.path.join(cluster_dir, "train", "images")
        val_img_dir = os.path.join(cluster_dir, "val", "images")
        test_img_dir = os.path.join(cluster_dir, "test", "images")
        train_label_dir = os.path.join(cluster_dir, "train", "labels")
        val_label_dir = os.path.join(cluster_dir, "val", "labels")
        test_label_dir = os.path.join(cluster_dir, "test", "labels")

        # Création des sous-dossiers
        for path in [train_img_dir, val_img_dir, test_img_dir, train_label_dir, val_label_dir, test_label_dir]:
            os.makedirs(path, exist_ok=True)

        # Mélanger les images pour une répartition aléatoire
        random.shuffle(image_list)
        
        # Calcul des indices de split pour train, val et test
        n_total = len(image_list)
        n_train = int(0.7 * n_total)  # 70% pour train
        n_val = int(0.15 * n_total)   # 15% pour val
        n_test = n_total - n_train - n_val  # 15% pour test

        # Diviser les images en train, val et test
        train_images = image_list[:n_train]
        val_images = image_list[n_train:n_train + n_val]
        test_images = image_list[n_train + n_val:]

        # Copier les fichiers dans les dossiers correspondants
        def move_files(image_list, dest_images_dir, dest_labels_dir):
            for image in image_list:
                # Copier le fichier image
                shutil.copy(os.path.join(output_image_dir, image), dest_images_dir)
                
                # Copier le fichier d'annotation associé depuis le bon dossier (yolo5label_dir)
                label_file = image.replace('.jpg', '.txt').replace('.png', '.txt')
                label_path = os.path.join(yolo5label_dir, label_file)  # Correctement chercher dans yolo5label_dir
                if os.path.isfile(label_path):
                    shutil.copy(label_path, dest_labels_dir)
                else:
                    print(f"Attention : le fichier d'annotation pour {image} est introuvable dans {yolo5label_dir}.")

        # Appliquer le déplacement pour chaque split
        move_files(train_images, train_img_dir, train_label_dir)
        move_files(val_images, val_img_dir, val_label_dir)
        move_files(test_images, test_img_dir, test_label_dir)

        print(f"Cluster {cluster_label}: train ({len(train_images)}), val ({len(val_images)}), test ({len(test_images)}) répartis.")

# Exécuter la fonction pour copier et diviser les fichiers
copy_files_by_cluster(clusters)

print("Séparation des images par clusters et par splits train, val, test terminée.")


Cluster 5: train (318), val (68), test (69) répartis.
Cluster 0: train (87), val (18), test (20) répartis.
Cluster 2: train (340), val (73), test (74) répartis.
Cluster 3: train (147), val (31), test (32) répartis.
Cluster 1: train (169), val (36), test (37) répartis.
Cluster 6: train (125), val (26), test (28) répartis.
Cluster 4: train (80), val (17), test (18) répartis.
Séparation des images par clusters et par splits train, val, test terminée.


In [6]:
import os

nc = 2  # Nombre de classes
names = ['label', 'pal']  # Noms des classes

# Fonction pour créer un fichier dataset.yaml pour chaque cluster
def create_yaml_for_cluster(cluster_label):
    cluster_dir = os.path.join(output_base_dir, f"cluster_{cluster_label}")
    yaml_content = f"""
# Dataset paths for cluster {cluster_label}
train: ..\\{os.path.join('dataset', 'cluster', f'cluster_{cluster_label}', 'train', 'images').replace(os.sep, '\\\\')}
val: ..\\{os.path.join('dataset', 'cluster', f'cluster_{cluster_label}', 'val', 'images').replace(os.sep, '\\\\')}
test: ..\\{os.path.join('dataset', 'cluster', f'cluster_{cluster_label}', 'test', 'images').replace(os.sep, '\\\\')}

# Number of classes
nc: {nc}

# Names of classes
names: {names}
"""
    # Sauvegarde du fichier YAML pour le cluster
    yaml_path = os.path.join(cluster_dir, "dataset.yaml")
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(yaml_content.strip())
    print(f"Fichier dataset.yaml créé pour le cluster {cluster_label}")

for cluster_label in range(n_clusters):  # Ajuster en fonction du nombre réel de clusters
    create_yaml_for_cluster(cluster_label)


Fichier dataset.yaml créé pour le cluster 0
Fichier dataset.yaml créé pour le cluster 1
Fichier dataset.yaml créé pour le cluster 2
Fichier dataset.yaml créé pour le cluster 3
Fichier dataset.yaml créé pour le cluster 4
Fichier dataset.yaml créé pour le cluster 5
Fichier dataset.yaml créé pour le cluster 6


In [ ]:
# Fonction pour générer et afficher les commandes pour YOLOv5
def print_yolov5_commands_for_cluster(cluster_label):
    # Chemin relatif vers le fichier dataset.yaml du cluster
    yaml_filepath = os.path.join("..", "dataset", "cluster", f"cluster_{cluster_label}", "dataset.yaml")
    
    # Construire la commande pour YOLOv5
    command = [
        "python", "train.py",
        "--img", "640",
        "--batch-size", "16",
        "--epochs", "50",
        "--data", yaml_filepath,
        "--weights", "yolov5s.pt",
        "--project", f"model2_yolo5_cluster_{cluster_label}",
        "--name", f"experiment_cluster_{cluster_label}",
        "--exist-ok",
        "--device", "cpu",
        "--save-period", "5"
    ]
    
    # Afficher la commande à exécuter
    print(f"Commande à exécuter pour le cluster {cluster_label} :")
    print(" ".join(command))
    print()

# Parcourir les clusters et afficher les commandes pour chaque cluster
for cluster_label in range(n_clusters):
    print_yolov5_commands_for_cluster(cluster_label)

print("exécutez les commandes dans le terminal où se situe le yolov5.")

Commande à exécuter pour le cluster 0 :
python train.py --img 640 --batch-size 16 --epochs 50 --data ..\dataset\cluster\cluster_0\dataset.yaml --weights yolov5s.pt --project model2_yolo5_cluster_0 --name experiment_cluster_0 --exist-ok --device cpu --save-period 5

Commande à exécuter pour le cluster 1 :
python train.py --img 640 --batch-size 16 --epochs 50 --data ..\dataset\cluster\cluster_1\dataset.yaml --weights yolov5s.pt --project model2_yolo5_cluster_1 --name experiment_cluster_1 --exist-ok --device cpu --save-period 5

Commande à exécuter pour le cluster 2 :
python train.py --img 640 --batch-size 16 --epochs 50 --data ..\dataset\cluster\cluster_2\dataset.yaml --weights yolov5s.pt --project model2_yolo5_cluster_2 --name experiment_cluster_2 --exist-ok --device cpu --save-period 5

Commande à exécuter pour le cluster 3 :
python train.py --img 640 --batch-size 16 --epochs 50 --data ..\dataset\cluster\cluster_3\dataset.yaml --weights yolov5s.pt --project model2_yolo5_cluster_3 --nam

faire un code qui print juste les commandes à faire en vrai :

python train.py --img 640 --batch-size 16 --epochs 50 --data "../dataset/cluster/cluster_0/dataset.yaml" --weights yolov5s.pt --project "model2_yolo5_cluster_0" --name "experiment_cluster_0" --exist-ok --device cpu --save-period 5